In [1]:
#https://www.kaggle.com/daisukelab/creating-fat2019-preprocessed-data/output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import librosa
import librosa.display
import random
import os
from tqdm import tqdm 

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence


from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping 
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv2D, MaxPooling2D, Dropout, Activation, Input,BatchNormalization, AveragePooling2D,GlobalMaxPool2D,PReLU

from tensorflow.keras.models import model_from_json  
from tensorflow.keras.applications import DenseNet169


from keras_preprocessing.image import ImageDataGenerator

In [2]:
curated_data = pd.read_csv("../input/sc2finalsub/train_curated.csv")

In [3]:
curated_data.shape

(4970, 2)

# Creating CSV data based on input .wav file

In [4]:
temp_label=[]
temp_fname=[]

for file in os.listdir("../input/sc2finalsub/train_curated 100/train_curated"):
   
    if file=="desktop.ini":
       continue
    temp_fname.append(file)
    if len(file.split("_"))==1:
         temp=file.split(".")[0]+".wav"
    elif len(file.split("_"))==2:  
        temp=file.split("_")[0]+".wav" 
    try:   
        label=curated_data[curated_data["fname"]==temp].iloc[0]["labels"]
    except IndexError:    
        label=curated_data[curated_data["fname"]==temp].iloc[0]["labels"]
    temp_label.append(label)

In [5]:
data=pd.DataFrame({"fname":temp_fname,"labels":temp_label})

## Setting the paarameters for Spectrogaram

In [6]:
!mkdir ./sub

## Functions for loading .wav files and converting to Specrogram images

In [7]:

#Top Notebook in cometition to preprocess:https://www.kaggle.com/daisukelab/cnn-2d-basic-solution-powered-by-fast-ai

class conf:
    sampling_rate = 44100
    duration = 2 # sec
    hop_length = 347*duration # to make time steps 128
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    padmode = 'constant'
    samples = sampling_rate * duration

In [8]:
def read_audio(conf, pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=conf.sampling_rate)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf.samples: # long enough
        if trim_long_data:
            y = y[0:0+conf.samples]
    else: # pad blank
        padding = conf.samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), conf.padmode)
    return y



def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram


def show_melspectrogram(conf, mels, title='Log-frequency power spectrogram'):
    librosa.display.specshow(mels, x_axis='time', y_axis='mel', 
                             sr=conf.sampling_rate, hop_length=conf.hop_length,
                            fmin=conf.fmin, fmax=conf.fmax)
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.show()


def read_as_melspectrogram(conf, pathname, trim_long_data, debug_display=False):
    x = read_audio(conf, pathname, trim_long_data)
    mels = audio_to_melspectrogram(conf, x)
    if debug_display:
        IPython.display.display(IPython.display.Audio(x, rate=conf.sampling_rate))
        show_melspectrogram(conf, mels)
    return mels

def convert_wav_to_image(df, source):
    X = []
    for i, row in tqdm_notebook(df.iterrows()):
        x = read_as_melspectrogram(conf, source/str(row.fname), trim_long_data=False)
        x_color = mono_to_color(x)
        X.append(x_color)
    return X

def get_default_conf():
    return conf


In [9]:
def create_spectrogram(x,name,data_type):
    plt.interactive(False)
    #clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    librosa.display.specshow(x, x_axis='time', y_axis='mel', 
                             sr=conf.sampling_rate, hop_length=conf.hop_length,
                            fmin=conf.fmin, fmax=conf.fmax)
    #show_melspectrogram(conf, x)
    filename  = Path('./'+data_type+'/' + name + '.jpg')
    fig.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del name,fig,ax 

## final_fun_1 defination which will return predictions of input .wav files

In [10]:
def final_fun_1(DIR):
    
    raw_data=os.listdir(DIR)

    conf = get_default_conf()
    
    
    for fileName in tqdm(raw_data):
             if fileName=="desktop.ini":
                continue
             x=read_as_melspectrogram(conf, DIR+fileName, trim_long_data=True,debug_display=False)
             create_spectrogram(x,fileName.split(".")[0],"./sub/")
    
    
    ## Loading Images
    test_datagen=ImageDataGenerator(rescale=1./255.)
    test_generator=test_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({"fname":[i.split(".")[0]+".jpg"for i in os.listdir(DIR)]}),
        directory="./sub/",
        x_col="fname",
        y_col=None,
        batch_size=64,
        seed=42,
        shuffle=False,
        class_mode=None,
        target_size=(128,128))
    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    
    
    
    
    
    # Loading Model
    image_input = Input(shape=(128,128, 3))
    model = DenseNet169(input_tensor=image_input, include_top=True)
    last_layer = model.get_layer('avg_pool').output
    x= Flatten(name='flatten')(last_layer)
    x= Dense(80)(x)
    output = Activation('sigmoid')(x)
    Final_model = Model(inputs=image_input,outputs= output)
    Final_model.load_weights("../input/sc2finalsub/DenseNetTotal.best_weights.hdf5")
    
    test_generator.reset()
    res_test=Final_model.predict_generator(test_generator,verbose=1)
    
    return res_test
    

In [11]:
DIR="../input/sc2finalsub/train_curated 100/train_curated/"
result=final_fun_1(DIR)

100%|██████████| 101/101 [00:11<00:00,  9.10it/s]
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="fname". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 100 validated image filenames.
2/2 [==============================] - 1s 536ms/step


In [12]:
result.shape

(100, 80)

In [13]:
import pickle
labels = pickle.load( open( "../input/sc2finalsub/labels.pkl", "rb" ) )

In [14]:
index=82
temp_labels=[labels["labels"][j] for j,i in enumerate(list(result[index])) if i>=0.5]
if len(temp_labels)==0:
   temp=labels["labels"][np.argmax(result[index])]
   print(temp)
else:
    print(temp_labels)

['Drip', 'Raindrop']


## Result of final_fun_1

In [15]:
index=74
temp_labels=[labels["labels"][j] for j,i in enumerate(list(result[index])) if i>=0.5]
if len(temp_labels)==0:
   temp=labels["labels"][np.argmax(result[index])]
   print(temp)
else:
    print(temp_labels)

['Cupboard_open_or_close', 'Squeak']


In [16]:
index=71
temp_labels=[labels["labels"][j] for j,i in enumerate(list(result[index])) if i>=0.5]
if len(temp_labels)==0:
   temp=labels["labels"][np.argmax(result[index])]
   print(temp)
else:
    print(temp_labels)

['Fill_(with_liquid)', 'Sink_(filling_or_washing)']


In [17]:

def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in target_columns:
        convert_to_bool(bool_df, col)
    return bool_df

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_train = labels["mlb_train"]
labels_test = mlb_train.transform([ i.split(",") for i in list(data["labels"])])


In [19]:
labels_test[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
testmultidf=pd.DataFrame(data=labels_test,columns=list(mlb_train.classes_))
testmultidf["fname"]=list(data["fname"])

In [21]:
target_columns=list(testmultidf.columns)[:-1]
train_bool = convert_dataframe_to_bool(testmultidf) 
train_lable_bool=train_bool[list(train_bool.columns)[:-1]].to_numpy()

In [22]:
train_lable_bool

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

# Evaluation Metric

In [23]:
import numpy as np
import sklearn.metrics

In [24]:
# All-in-one calculation of per-class lwlrap.

def calculate_per_class_lwlrap(truth, scores):
  """Calculate label-weighted label-ranking average precision.
  
  Arguments:
    truth: np.array of (num_samples, num_classes) giving boolean ground-truth
      of presence of that class in that sample.
    scores: np.array of (num_samples, num_classes) giving the classifier-under-
      test's real-valued score for each class for each sample.
  
  Returns:
    per_class_lwlrap: np.array of (num_classes,) giving the lwlrap for each 
      class.
    weight_per_class: np.array of (num_classes,) giving the prior of each 
      class within the truth labels.  Then the overall unbalanced lwlrap is 
      simply np.sum(per_class_lwlrap * weight_per_class)
  """
  assert truth.shape == scores.shape
  num_samples, num_classes = scores.shape
  # Space to store a distinct precision value for each class on each sample.
  # Only the classes that are true for each sample will be filled in.
  precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
  for sample_num in range(num_samples):
    pos_class_indices, precision_at_hits = (
      _one_sample_positive_class_precisions(scores[sample_num, :], 
                                            truth[sample_num, :]))
    precisions_for_samples_by_classes[sample_num, pos_class_indices] = (
        precision_at_hits)
  labels_per_class = np.sum(truth > 0, axis=0)
  weight_per_class = labels_per_class / float(np.sum(labels_per_class))
  # Form average of each column, i.e. all the precisions assigned to labels in
  # a particular class.
  per_class_lwlrap = (np.sum(precisions_for_samples_by_classes, axis=0) / 
                      np.maximum(1, labels_per_class))
  # overall_lwlrap = simple average of all the actual per-class, per-sample precisions
  #                = np.sum(precisions_for_samples_by_classes) / np.sum(precisions_for_samples_by_classes > 0)
  #           also = weighted mean of per-class lwlraps, weighted by class label prior across samples
  #                = np.sum(per_class_lwlrap * weight_per_class)
  return per_class_lwlrap, weight_per_class

In [25]:
# Calculate the overall lwlrap using sklearn.metrics function.

def calculate_overall_lwlrap_sklearn(truth, scores):
  """Calculate the overall lwlrap using sklearn.metrics.lrap."""
  # sklearn doesn't correctly apply weighting to samples with no labels, so just skip them.
  sample_weight = np.sum(truth > 0, axis=1)
  nonzero_weight_sample_indices = np.flatnonzero(sample_weight > 0)
  overall_lwlrap = sklearn.metrics.label_ranking_average_precision_score(
      truth[nonzero_weight_sample_indices, :] > 0, 
      scores[nonzero_weight_sample_indices, :], 
      sample_weight=sample_weight[nonzero_weight_sample_indices])
  return overall_lwlrap

In [26]:
# Accumulator object version.

class lwlrap_accumulator(object):
  """Accumulate batches of test samples into per-class and overall lwlrap."""  

  def __init__(self):
    self.num_classes = 0
    self.total_num_samples = 0
  
  def accumulate_samples(self, batch_truth, batch_scores):
    """Cumulate a new batch of samples into the metric.
    
    Args:
      truth: np.array of (num_samples, num_classes) giving boolean
        ground-truth of presence of that class in that sample for this batch.
      scores: np.array of (num_samples, num_classes) giving the 
        classifier-under-test's real-valued score for each class for each
        sample.
    """
    assert batch_scores.shape == batch_truth.shape
    num_samples, num_classes = batch_truth.shape
    if not self.num_classes:
      self.num_classes = num_classes
      self._per_class_cumulative_precision = np.zeros(self.num_classes)
      self._per_class_cumulative_count = np.zeros(self.num_classes, 
                                                  dtype=np.int)
    assert num_classes == self.num_classes
    for truth, scores in zip(batch_truth, batch_scores):
      pos_class_indices, precision_at_hits = (
        _one_sample_positive_class_precisions(scores, truth))
      self._per_class_cumulative_precision[pos_class_indices] += (
        precision_at_hits)
      self._per_class_cumulative_count[pos_class_indices] += 1
    self.total_num_samples += num_samples

  def per_class_lwlrap(self):
    """Return a vector of the per-class lwlraps for the accumulated samples."""
    return (self._per_class_cumulative_precision / 
            np.maximum(1, self._per_class_cumulative_count))

  def per_class_weight(self):
    """Return a normalized weight vector for the contributions of each class."""
    return (self._per_class_cumulative_count / 
            float(np.sum(self._per_class_cumulative_count)))

  def overall_lwlrap(self):
    """Return the scalar overall lwlrap for cumulated samples."""
    return np.sum(self.per_class_lwlrap() * self.per_class_weight())

## final_fun_2 defination which will return Scores of input .wav files

In [27]:
def final_fun_2(DIR,Y):
    truth = Y
    scores = final_fun_1(DIR)
    return calculate_overall_lwlrap_sklearn(truth, scores)

In [28]:
DIR="../input/sc2finalsub/train_curated 100/train_curated/"
scores=final_fun_2(DIR,train_lable_bool)

100%|██████████| 101/101 [00:10<00:00,  9.94it/s]
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1 invalid image filename(s) in x_col="fname". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 100 validated image filenames.
2/2 [==============================] - 1s 707ms/step


## Result of final_fun_2  

In [29]:
scores

0.9443311766841179